In [7]:
import os
import sys
import torch
import numpy as np
import trimesh
import datetime
import logging
import provider
import importlib
import shutil
import argparse
from dotenv import load_dotenv
load_dotenv()

from pathlib import Path
from tqdm import tqdm
# from data_utils.ModelNetDataLoader import ModelNetDataLoader

DATASET_DIR = os.getenv('DATASET_DIR')
print(DATASET_DIR)
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
# ROOT_DIR = BASE_DIR
# sys.path.append(os.path.join(ROOT_DIR, 'models'))

D:\Datasets\ModelNet\ModelNet10


# Generating Names for Objects

In [5]:
import os

DATASET_DIR = "D:/Datasets/ModelNet/ModelNet10"
for split in ["train", "test"]:
    OUTPUT_FILE = os.path.join(DATASET_DIR, f"modelnet10_{split}.txt")

    with open(OUTPUT_FILE, 'w') as f_out:
        for class_dir in sorted(os.listdir(DATASET_DIR)):
            class_path = os.path.join(DATASET_DIR, class_dir, split)
            if os.path.isdir(class_path):
                for file in os.listdir(class_path):
                    if file.endswith('.off') or file.endswith('.txt'):
                        file_id = os.path.splitext(file)[0]
                        f_out.write(f"{file_id}\n")

    print(f"✅ Saved: {OUTPUT_FILE}")


✅ Saved: D:/Datasets/ModelNet/ModelNet10\modelnet10_train.txt
✅ Saved: D:/Datasets/ModelNet/ModelNet10\modelnet10_test.txt


# Generating .txt point samples

In [8]:
NUM_POINTS = 1024

# Function to convert .off to .txt (point cloud)
def sample_off_to_txt(file_path, dest_path, num_points=1024):
    try:
        mesh = trimesh.load(file_path)
        if not mesh.is_watertight:
            mesh = mesh.convex_hull

        points, _ = trimesh.sample.sample_surface(mesh, num_points)
        np.savetxt(dest_path, points, fmt='%.6f', delimiter=',')
        return True
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return False

# Process both train and test splits
for split in ["train", "test"]:
    output_txt = os.path.join(DATASET_DIR, f"modelnet10_{split}.txt")

    with open(output_txt, 'w') as f_out:
        for class_dir in sorted(os.listdir(DATASET_DIR)):
            class_path = os.path.join(DATASET_DIR, class_dir, split)
            if os.path.isdir(class_path):
                for file in sorted(os.listdir(class_path)):
                    if file.endswith('.off'):
                        file_id = os.path.splitext(file)[0]
                        off_path = os.path.join(class_path, file)
                        txt_path = os.path.join(class_path, file_id + '.txt')

                        # Convert if .txt does not exist
                        if not os.path.exists(txt_path):
                            success = sample_off_to_txt(off_path, txt_path, NUM_POINTS)
                            if not success:
                                continue  # skip writing to the file list if conversion failed

                        # Add entry to split list
                        f_out.write(f"{file_id}\n")